## Abstract

The goal of this assignment is to demonstrate how to capture unconventional datasets, clean and store them. We'll collect data from Wikipedia about the top cities in the United States. The final format will be a CSV file that is ready to be uploaded to a BigQuery table

## Table of Contents

1. <a href='#acquiring_Data'>Acquiring the Data</a>
2. <a href='#processing_Data'>Processing the Data</a>
3. <a href='#storing_Data'>Storing the Data</a>

<a id='acquiring_Data'></a>
## Acquiring the Data

To start, we need to scrap the data we are going to use for our system. The datasets we are going to use is from [Wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population) , about the top cities in the United States.

In [24]:
# Import Libraries
import pandas as pd
import wikipedia as wp
import warnings
warnings.filterwarnings('ignore')

In [157]:
#Get the html source
html = wp.page("List of United States cities by population").html().encode("UTF-8")
df = pd.read_html(html)[4]

In [158]:
print (df)

           0                    1                     2             3   \
0    2018rank                 City              State[c]  2018estimate   
1           1     New York City[d]              New York       8398748   
2           2          Los Angeles            California       3990456   
3           3              Chicago              Illinois       2705994   
4           4           Houston[3]                 Texas       2325502   
5           5              Phoenix               Arizona       1660272   
6           6      Philadelphia[e]          Pennsylvania       1584138   
7           7          San Antonio                 Texas       1532233   
8           8            San Diego            California       1425976   
9           9               Dallas                 Texas       1345047   
10         10             San Jose            California       1030119   
11         11               Austin                 Texas        964254   
12         12      Jacksonville[f]    

<a id='processing_Data'></a>
## Processing the Data

With the datasets in place, let's now start formatting it.

In [159]:
# Setting the first row of our dataframe as column header
df.columns = df.iloc[0]

In [160]:
# As the original wikipedia table contains colspan hence we have to format our dataframe
df['2016_land_area'] = df['2016 land area'] +' '+ df['2016 population density']
df['2016_Population_Density'] = df['Location'] +' '+ df.iloc[ : , 9]
df.drop(df.columns[[6, 7, 8, 9]], axis = 1, inplace = True)

In [161]:
# Replacing superscript character present in the City name
df['City'] = df['City'].str.replace(r"\[.*\]","")

In [162]:
# Dropping the first row from our dataframe as it is contains duplicate column header
wiki_df = df.drop(df.index[0])

In [164]:
# Renaming column name according to wikipedia table
wiki_df.rename(columns={'2018rank': '2017rank', 'State[c]':'State', '2018estimate': '2017estimate'}, inplace=True)

In [165]:
wiki_df

,2017rank,City,State,2017estimate,2010Census,Change,2016_land_area,2016_Population_Density
1,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi 780.9 km2,"28,317/sq mi 10,933/km2"
2,2,Los Angeles,California,3990456,3792621,+5.22%,"468.7 sq mi 1,213.9 km2","8,484/sq mi 3,276/km2"
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi 588.7 km2,"11,900/sq mi 4,600/km2"
4,4,Houston,Texas,2325502,2100263,+10.72%,"637.5 sq mi 1,651.1 km2","3,613/sq mi 1,395/km2"
5,5,Phoenix,Arizona,1660272,1445632,+14.85%,"517.6 sq mi 1,340.6 km2","3,120/sq mi 1,200/km2"
6,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2 sq mi 347.6 km2,"11,683/sq mi 4,511/km2"
7,7,San Antonio,Texas,1532233,1327407,+15.43%,"461.0 sq mi 1,194.0 km2","3,238/sq mi 1,250/km2"
8,8,San Diego,California,1425976,1307402,+9.07%,325.2 sq mi 842.3 km2,"4,325/sq mi 1,670/km2"
9,9,Dallas,Texas,1345047,1197816,+12.29%,340.9 sq mi 882.9 km2,"3,866/sq mi 1,493/km2"
10,10,San Jose,California,1030119,945942,+8.90%,177.5 sq mi 459.7 km2,"5,777/sq mi 2,231/km2"


In [166]:
# The below function will extract additional information from each city page. In our case it will be the first sentence on each city page
def add_info(value):
    ny = wp.page(value)
    ny1 = wp.summary(value, sentences=1)
    index = value.index
    return ny.url,ny1

In [167]:
# Creating tuple to store wikipedia url of the city and the first sentence on the city page
str = ();
list=[]
for index, row in wiki_df.iterrows():
    try:
        str= str + add_info(row['City'])
        list.append(index)
#         print((add_info(row['City'])))
    except wp.exceptions.DisambiguationError as e:
        pass

In [173]:
# Dropping all the rows where Cityname Url resulted in DisambiguationError above
wiki_df = wiki_df[wiki_df.index.isin(list)]

In [174]:
wiki_df

,2017rank,City,State,2017estimate,2010Census,Change,2016_land_area,2016_Population_Density
1,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi 780.9 km2,"28,317/sq mi 10,933/km2"
2,2,Los Angeles,California,3990456,3792621,+5.22%,"468.7 sq mi 1,213.9 km2","8,484/sq mi 3,276/km2"
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi 588.7 km2,"11,900/sq mi 4,600/km2"
4,4,Houston,Texas,2325502,2100263,+10.72%,"637.5 sq mi 1,651.1 km2","3,613/sq mi 1,395/km2"
6,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2 sq mi 347.6 km2,"11,683/sq mi 4,511/km2"
7,7,San Antonio,Texas,1532233,1327407,+15.43%,"461.0 sq mi 1,194.0 km2","3,238/sq mi 1,250/km2"
8,8,San Diego,California,1425976,1307402,+9.07%,325.2 sq mi 842.3 km2,"4,325/sq mi 1,670/km2"
9,9,Dallas,Texas,1345047,1197816,+12.29%,340.9 sq mi 882.9 km2,"3,866/sq mi 1,493/km2"
10,10,San Jose,California,1030119,945942,+8.90%,177.5 sq mi 459.7 km2,"5,777/sq mi 2,231/km2"
11,11,Austin,Texas,964254,790390,+22.00%,312.7 sq mi 809.9 km2,"3,031/sq mi 1,170/km2"


In [175]:
# Converting above generated tuple to dataframe
str = pd.Series(str)
addInfoCity_df = pd.DataFrame(data=str.values, columns=['add_value'])

In [176]:
addInfoCity_df

,add_value
0,https://en.wikipedia.org/wiki/New_York_City
1,"The City of New York, usually called either Ne..."
2,https://en.wikipedia.org/wiki/Los_Angeles
3,"Los Angeles ( (listen); Spanish: Los Ángeles),..."
4,https://en.wikipedia.org/wiki/Chicago
5,"Chicago ( (listen), locally also ), officially..."
6,https://en.wikipedia.org/wiki/Houston
7,Houston ( (listen) HEW-stən) is the most popul...
8,https://en.wikipedia.org/wiki/Philadelphia
9,"Philadelphia (Ancient Greek: φίλος ἀδελφός), k..."


In [177]:
# Creating a dataframe to store additional info of individual city(i.e. URL and first sentence on each city wikipedia page).
# Later we'll merge this dataframe with our original dataframe
CityUrl = addInfoCity_df.iloc[range(0,len(addInfoCity_df),2)]
InfoCity = addInfoCity_df.iloc[range(1,len(addInfoCity_df),2)]
CityUrl = CityUrl.reset_index(drop=True)
InfoCity = InfoCity.reset_index(drop=True)
City = pd.merge(CityUrl, InfoCity, left_index=True, right_index=True)
City.columns = ['CityURL', 'CityInfo']

In [178]:
# Changing City dataframe index to start with 1
City.index += 1

In [179]:
City

,CityURL,CityInfo
1,https://en.wikipedia.org/wiki/New_York_City,"The City of New York, usually called either Ne..."
2,https://en.wikipedia.org/wiki/Los_Angeles,"Los Angeles ( (listen); Spanish: Los Ángeles),..."
3,https://en.wikipedia.org/wiki/Chicago,"Chicago ( (listen), locally also ), officially..."
4,https://en.wikipedia.org/wiki/Houston,Houston ( (listen) HEW-stən) is the most popul...
5,https://en.wikipedia.org/wiki/Philadelphia,"Philadelphia (Ancient Greek: φίλος ἀδελφός), k..."
6,https://en.wikipedia.org/wiki/San_Antonio,"San Antonio (; from Spanish, ""Saint Anthony""),..."
7,https://en.wikipedia.org/wiki/San_Diego,"San Diego (; Spanish for ""Saint Didacus""; Span..."
8,https://en.wikipedia.org/wiki/Dallas,"Dallas (), officially the City of Dallas, is a..."
9,"https://en.wikipedia.org/wiki/San_Jose,_Califo...",San Jose (; Spanish for 'Saint Joseph'; Spanis...
10,"https://en.wikipedia.org/wiki/Austin,_Texas",Austin is the capital of the U.S. state of Tex...


In [180]:
# Resetting wiki_df dataframe and changing index value to start with 1
wiki_df= wiki_df.reset_index(drop=True)
wiki_df.index += 1

In [181]:
wiki_df

,2017rank,City,State,2017estimate,2010Census,Change,2016_land_area,2016_Population_Density
1,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi 780.9 km2,"28,317/sq mi 10,933/km2"
2,2,Los Angeles,California,3990456,3792621,+5.22%,"468.7 sq mi 1,213.9 km2","8,484/sq mi 3,276/km2"
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi 588.7 km2,"11,900/sq mi 4,600/km2"
4,4,Houston,Texas,2325502,2100263,+10.72%,"637.5 sq mi 1,651.1 km2","3,613/sq mi 1,395/km2"
5,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2 sq mi 347.6 km2,"11,683/sq mi 4,511/km2"
6,7,San Antonio,Texas,1532233,1327407,+15.43%,"461.0 sq mi 1,194.0 km2","3,238/sq mi 1,250/km2"
7,8,San Diego,California,1425976,1307402,+9.07%,325.2 sq mi 842.3 km2,"4,325/sq mi 1,670/km2"
8,9,Dallas,Texas,1345047,1197816,+12.29%,340.9 sq mi 882.9 km2,"3,866/sq mi 1,493/km2"
9,10,San Jose,California,1030119,945942,+8.90%,177.5 sq mi 459.7 km2,"5,777/sq mi 2,231/km2"
10,11,Austin,Texas,964254,790390,+22.00%,312.7 sq mi 809.9 km2,"3,031/sq mi 1,170/km2"


In [182]:
# Merging wiki_df and City dataframes
df_out = pd.merge(wiki_df, City, left_index=True, right_index=True)

In [183]:
df_out

,2017rank,City,State,2017estimate,2010Census,Change,2016_land_area,2016_Population_Density,CityURL,CityInfo
1,1,New York City,New York,8398748,8175133,+2.74%,301.5 sq mi 780.9 km2,"28,317/sq mi 10,933/km2",https://en.wikipedia.org/wiki/New_York_City,"The City of New York, usually called either Ne..."
2,2,Los Angeles,California,3990456,3792621,+5.22%,"468.7 sq mi 1,213.9 km2","8,484/sq mi 3,276/km2",https://en.wikipedia.org/wiki/Los_Angeles,"Los Angeles ( (listen); Spanish: Los Ángeles),..."
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3 sq mi 588.7 km2,"11,900/sq mi 4,600/km2",https://en.wikipedia.org/wiki/Chicago,"Chicago ( (listen), locally also ), officially..."
4,4,Houston,Texas,2325502,2100263,+10.72%,"637.5 sq mi 1,651.1 km2","3,613/sq mi 1,395/km2",https://en.wikipedia.org/wiki/Houston,Houston ( (listen) HEW-stən) is the most popul...
5,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2 sq mi 347.6 km2,"11,683/sq mi 4,511/km2",https://en.wikipedia.org/wiki/Philadelphia,"Philadelphia (Ancient Greek: φίλος ἀδελφός), k..."
6,7,San Antonio,Texas,1532233,1327407,+15.43%,"461.0 sq mi 1,194.0 km2","3,238/sq mi 1,250/km2",https://en.wikipedia.org/wiki/San_Antonio,"San Antonio (; from Spanish, ""Saint Anthony""),..."
7,8,San Diego,California,1425976,1307402,+9.07%,325.2 sq mi 842.3 km2,"4,325/sq mi 1,670/km2",https://en.wikipedia.org/wiki/San_Diego,"San Diego (; Spanish for ""Saint Didacus""; Span..."
8,9,Dallas,Texas,1345047,1197816,+12.29%,340.9 sq mi 882.9 km2,"3,866/sq mi 1,493/km2",https://en.wikipedia.org/wiki/Dallas,"Dallas (), officially the City of Dallas, is a..."
9,10,San Jose,California,1030119,945942,+8.90%,177.5 sq mi 459.7 km2,"5,777/sq mi 2,231/km2","https://en.wikipedia.org/wiki/San_Jose,_Califo...",San Jose (; Spanish for 'Saint Joseph'; Spanis...
10,11,Austin,Texas,964254,790390,+22.00%,312.7 sq mi 809.9 km2,"3,031/sq mi 1,170/km2","https://en.wikipedia.org/wiki/Austin,_Texas",Austin is the capital of the U.S. state of Tex...


In [184]:
# Removing all non-ascii characters in the columns
def remove_nonascii(column_name):
    df_out[column_name] = df_out[column_name].str.replace(r'[^\x00-\x7f]', '')

In [185]:
remove_nonascii('2016_land_area')
remove_nonascii('2016_Population_Density')

In [186]:
df_out['Change'] = df_out['Change'].str.replace('â', '-')

In [187]:
df_out

,2017rank,City,State,2017estimate,2010Census,Change,2016_land_area,2016_Population_Density,CityURL,CityInfo
1,1,New York City,New York,8398748,8175133,+2.74%,301.5sqmi 780.9km2,"28,317/sqmi 10,933/km2",https://en.wikipedia.org/wiki/New_York_City,"The City of New York, usually called either Ne..."
2,2,Los Angeles,California,3990456,3792621,+5.22%,"468.7sqmi 1,213.9km2","8,484/sqmi 3,276/km2",https://en.wikipedia.org/wiki/Los_Angeles,"Los Angeles ( (listen); Spanish: Los Ángeles),..."
3,3,Chicago,Illinois,2705994,2695598,+0.39%,227.3sqmi 588.7km2,"11,900/sqmi 4,600/km2",https://en.wikipedia.org/wiki/Chicago,"Chicago ( (listen), locally also ), officially..."
4,4,Houston,Texas,2325502,2100263,+10.72%,"637.5sqmi 1,651.1km2","3,613/sqmi 1,395/km2",https://en.wikipedia.org/wiki/Houston,Houston ( (listen) HEW-stən) is the most popul...
5,6,Philadelphia,Pennsylvania,1584138,1526006,+3.81%,134.2sqmi 347.6km2,"11,683/sqmi 4,511/km2",https://en.wikipedia.org/wiki/Philadelphia,"Philadelphia (Ancient Greek: φίλος ἀδελφός), k..."
6,7,San Antonio,Texas,1532233,1327407,+15.43%,"461.0sqmi 1,194.0km2","3,238/sqmi 1,250/km2",https://en.wikipedia.org/wiki/San_Antonio,"San Antonio (; from Spanish, ""Saint Anthony""),..."
7,8,San Diego,California,1425976,1307402,+9.07%,325.2sqmi 842.3km2,"4,325/sqmi 1,670/km2",https://en.wikipedia.org/wiki/San_Diego,"San Diego (; Spanish for ""Saint Didacus""; Span..."
8,9,Dallas,Texas,1345047,1197816,+12.29%,340.9sqmi 882.9km2,"3,866/sqmi 1,493/km2",https://en.wikipedia.org/wiki/Dallas,"Dallas (), officially the City of Dallas, is a..."
9,10,San Jose,California,1030119,945942,+8.90%,177.5sqmi 459.7km2,"5,777/sqmi 2,231/km2","https://en.wikipedia.org/wiki/San_Jose,_Califo...",San Jose (; Spanish for 'Saint Joseph'; Spanis...
10,11,Austin,Texas,964254,790390,+22.00%,312.7sqmi 809.9km2,"3,031/sqmi 1,170/km2","https://en.wikipedia.org/wiki/Austin,_Texas",Austin is the capital of the U.S. state of Tex...


<a id='storing_Data'></a>
## Storing the Data

Thats it. With all the formatting done, we are ready to write the data into csv file.

In [188]:
# Writing final result to csv file
export_csv = df_out.to_csv (r'./wikipedia_dataframe.csv', index = None, header=True)